In [1]:
%matplotlib inline
import numpy as np
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
import theano.tensor as T
import theano
import lasagne
import lasagne.nonlinearities as nonlin
import pylab
import pickle
import re
import os
import imageio
import time
from IPython.display import display, clear_output
import PIL

from net.vgg16 import build_model, set_weights
from net.process import preprocess, resize_img

--- Logging error ---
Traceback (most recent call last):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\logging\__init__.py", line 978, in emit
    msg = self.format(record)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\logging\__init__.py", line 828, in format
    return fmt.format(record)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\logging\__init__.py", line 573, in format
    record.exc_text = self.formatException(record.exc_info)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\logging\__init__.py", line 523, in formatException
    traceback.print_exception(ei[0], ei[1], tb, None, sio)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\traceback.py", line 169, in print_exception
    for line in _format_exception_iter(etype, value, tb, limit, chain):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\traceback.py", line 146, in _format_exception_iter
    for value, tb in values:
  File "E:\WinPython-64bit-3.4.4.1\python-3.4

In [2]:
dataDir='annotations_trainval2017'
dataType='val2017'
annFile='{}/annotations/captions_{}.json'.format(dataDir, dataType)

coco=COCO(annFile)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


In [3]:
print(len(coco.anns))
display(list(coco.anns.items())[-3:])

25014


[(541200,
  {'caption': 'a glass vase with some flower coming out of it ',
   'id': 541200,
   'image_id': 159282}),
 (262135,
  {'caption': 'A tennis player is taking a swing on a red court.',
   'id': 262135,
   'image_id': 551804}),
 (589821,
  {'caption': 'there is a yellow notebook on a black desk ',
   'id': 589821,
   'image_id': 176446})]

# Build and load VGG16

In [4]:
def make_network(name='net/vgg16.pkl'):
    net = build_model()
    net = set_weights(net, name='net/vgg16.pkl')
    
    input_image = T.tensor4('input')
    output_last = lasagne.layers.get_output(net['prob'], input_image)
    output_fc8 = lasagne.layers.get_output(net['fc8'], input_image)
    output_fc7 = lasagne.layers.get_output(net['fc7'], input_image)
    
    prob_and_vec = theano.function(
        [input_image], 
        [output_last, output_fc8, output_fc7])
    return net, prob_and_vec

In [5]:
# net, prob_and_vec = make_network()

# img2vec

In [6]:
from net.imgs import *
      
# %time imgs_process('annotations_trainval2017/img') # Wall time: 4h 43min 38s

In [7]:
imgs = imgs_load(dataDir)
display(len(imgs))

4991

In [8]:
temp = pickle.load(open('annotations_trainval2017\data\\130.data', 'rb'))
temp = list(temp.items())
print('max =', np.max(temp[10][1][1]))
print('min =', np.min(temp[10][1][1]))

max = 10.1163502519
min = 0.0


# texts

In [9]:
from net.sents_process import *

# sents_process(dict_from_anns(coco.anns), 'annotations_trainval2017')

In [10]:
anno = sent_load(dataDir)

In [11]:
danno = {}
for id, v in anno:
    danno[id] = danno.get(id, []) + [v]

In [12]:
captions = [text[1]['caption'] for text in coco.anns.items()]
display(captions[:5])

['Two mean are playing tennis and both are wearing sunglasses.  ',
 'a couple of people that are playing in a field',
 'A purple and white bus in a parking lot.',
 'a tennis player swinging a racket at a ball',
 'a person attempting a jump with a skateboard']

In [13]:
def mix_text_vec2vec(sent):
    only_vec = list(filter(lambda x: not isinstance(x, str), sent))
    matr = np.vstack(tuple(only_vec))
    return np.concatenate((
            np.mean(matr, axis=0), 
            np.max(matr, axis=0), 
            np.min(matr, axis=0)))

vects = {}
for id, val in anno:
    vects[id] = vects.get(id, []) + [mix_text_vec2vec(val)]

# neural networks

In [14]:
from net.train import *

# CNN

Generate X, y

In [15]:
H = 300
W = 14

In [16]:
train_size = 23300

X, y = [], []
for id, data in imgs.items():
    while id[0] == '0':
        id = id[1:]
    try:
        for ls in danno[id]:
            vects = []
            for elem in ls:
                if not isinstance(elem, str):
                    vects.append(elem)
            vects = np.pad(vects, [(0, W), (0, 0)], mode='constant', constant_values=0)[:W].T
            X.append([vects])
            y.append(data[1])
    except:
        pass

print(len(X))

X_train, y_train, X_val, y_val = X[:train_size], y[:train_size], X[train_size:], y[train_size:]

23968


In [17]:
#with open('data.net', 'wb') as fl:
#    pickle.dump((X_train, y_train, X_val, y_val), fl)

In [ ]:
def build_cnn(file_name=None):
    net = NeuralNet(must_have=[
            'input_shape', 
            'learning_rate', 
            'train_fun', 
            'loss_fun', 
            'loss_fun_det', 
            'predict_fun_det'])
    
    input_shape = [None, H, W]
    
    input_X = T.tensor3("input X", dtype='float32')
    target_y = T.matrix("target Y", dtype='float32')

    
    net['inp'] = lasagne.layers.InputLayer(input_shape, input_var=input_X)
    
    net['max'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.max)
    net['min'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.min)
    net['mean'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.mean)
    net['con_1'] = lasagne.layers.Conv1DLayer(net['inp'], num_filters=64, filter_size=2)
    
    net['gmax'] = lasagne.layers.GlobalPoolLayer(net['con_1'], pool_function=T.max)
    
    #net['resh'] = lasagne.layers.ReshapeLayer(net['con_1'], shape=([0], -1))
    
    net['merge'] = lasagne.layers.ConcatLayer((net['max'], net['min'], net['mean'], net['gmax']))
    
    net['dens'] = lasagne.layers.DenseLayer(net['merge'], num_units=200, nonlinearity=nonlin.leaky_rectify)
    
    net['last'] = lasagne.layers.DenseLayer(net['dens'], num_units=4096)
    
    
    y_predicted = lasagne.layers.get_output(net['last'])
    y_predicted_det = lasagne.layers.get_output(net['last'], deterministic=True)

    all_weights = lasagne.layers.get_all_params(net['last'], trainable=True)
    
    learning_rate = theano.shared(lasagne.utils.floatX(0.001))
    loss = lasagne.objectives.squared_error(target_y, y_predicted).mean()
    loss_det = lasagne.objectives.squared_error(target_y, y_predicted_det).mean()
    
    # loss = loss + lasagne.regularization.regularize_layer_params(net['last'], lasagne.regularization.l2) * 0.01
    updates = lasagne.updates.adam(loss, all_weights, learning_rate=learning_rate)
    
    train_fun = theano.function([input_X, target_y], loss, updates=updates, allow_input_downcast=True)
    loss_fun = theano.function([input_X, target_y], loss, allow_input_downcast=True)
    loss_fun_det = theano.function([input_X, target_y], loss_det, allow_input_downcast=True)
    predict_fun_det = theano.function([input_X], y_predicted_det, allow_input_downcast=True)
    
    if file_name:
        load_net(net['last'], file_name, dataDir)
        
    def proc_vec(v):
        res = []
        for vv in v:
            res.append(vv[0])
        return res
    
    def proc_1(fun):
        def temp(v):
            v = proc_vec(v)
            return fun(v)
        return temp

    def proc_2(fun):
        def temp(v, u):
            v = proc_vec(v)
            return fun(v, u)

        return temp

    net.input_shape = input_shape
    net.learning_rate = learning_rate
    net.train_fun = proc_2(train_fun)
    net.loss_fun = proc_2(loss_fun)
    net.loss_fun_det = proc_2(loss_fun_det)
    net.predict_fun_det = proc_1(predict_fun_det)
    
    return net.check()

In [ ]:
net = build_cnn('new_vers_2.net')

In [ ]:
for key,val in net.items():
    if isinstance(val,lasagne.layers.Layer):
        params = val.get_params()
        
        print (key, val.output_shape, [p.get_value().shape for p in params])

In [ ]:
training_c = Training(net, 40)
training_c.set_Xy(X_train, y_train, X_val, y_val)

In [ ]:
training_c.training()

In [ ]:
save_net(net['last'], 'new_vers_2.net')